In [158]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud, visualize_mesh
import random
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.utils import class_weight

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [159]:
num_vertices = 100

In [173]:
config = {
    'experiment_name': 'vertix_edge_overfitting',
    'device': 'cuda:0',  
    'is_overfit': True,
    'batch_size': 1,
    'resume_ckpt': False,
    'learning_rate': 1e-4,
    'max_epochs': 50000,
    'print_every_n': 1,
    'validate_every_n': 1,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'overfit',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2,
    'overfit_single': True,
    'vertix_checkpoint': 'runs/vertix_train_hungarian/model_best.ckpt'
}

In [174]:
dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "val", threshold=config["num_vertices"], num_trajectories=config["num_trajectories"])


In [175]:
dataset.filter_data()

Length of dataset: 32304
Filtering data ..


100%|███████████████████████████████████| 32304/32304 [00:06<00:00, 5146.52it/s]

Length of dataset: 5474


In [176]:
from inference.inference_vertix_edge import InferenceHandlerVertixEdgeModel

# create a handler for inference using a trained checkpoint
inferer = InferenceHandlerVertixEdgeModel('runs/vertix_edge_hungarian/model_best.ckpt', config["num_vertices"], config["feature_size"])

In [177]:
x_indices = []
y_indices = []

graph = np.ones((1,config["num_vertices"], config["num_vertices"],1))

for i in range(config["num_vertices"]):
    for j in range(config["num_vertices"]):
        x_indices.append(i)
        y_indices.append(j)

In [180]:
def visualize_random():
    random_sample = random.randint(0,len(dataset)-1)
    input_sdf, target_vertices, mask, target_edges, edges_adj = dataset[random_sample]
    faces = []

    for i in range(num_vertices):
        for j in range(num_vertices):
            for k in range(num_vertices):
                if target_edges[i][j] and target_edges[j][k]:
                    faces.append(np.array([i,j,k]).reshape(1,-1))
                    
                    
    faces = np.concatenate(faces,0)
                    
        
                    
    visualize_mesh(target_vertices, faces)
    
    output_pointcloud, edges = inferer.infer_single(input_sdf, mask, x_indices,y_indices,edges_adj)
    
    faces = []

    
    target_size = int(sum(mask))
    
    matched_edges = np.zeros((num_vertices,num_vertices))
                    
    cost = distance_matrix(output_pointcloud, target_vertices[:int(sum(mask))])
                        
    vertix_idx, target_idx = linear_sum_assignment(cost) 
    
    for i in range(target_size):
        for j in range(target_size):
            curr_v_1 = vertix_idx[i]
            curr_t_1 = target_idx[i]
            curr_v_2 = vertix_idx[j]
            curr_t_2 = target_idx[j]
            matched_edges[curr_v_1,curr_v_2] = target_edges[curr_t_1,curr_t_2]
            
    faces = []

    for i in range(num_vertices):
        for j in range(num_vertices):
            for k in range(num_vertices):
                if edges[i][j] and edges[j][k]:
                    faces.append(np.array([i,j,k]).reshape(1,-1))
    faces = np.concatenate(faces,0)
        
    
    visualize_mesh(output_pointcloud,faces)
    

In [194]:
visualize_random()

Output()

Output()

424

Output()

Output()

In [82]:
edges.shape

(100, 100)

In [83]:
edges.sum()

3708

In [84]:
edges

array([[1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 1, 1],
       ...,
       [1, 1, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 1, 1]])

In [86]:
visualize_pointcloud(output_pointcloud,point_size=0.01)

Output()

Output()

NameError: name 'algorithm' is not defined